In [ ]:
  pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
import os
import random
from collections import defaultdict
from IPython.core.interactiveshell import InteractiveShell

import numpy as np
import pandas as pd

import tqdm
from surprise import BaselineOnly, CoClustering, Dataset, NMF, Reader, SlopeOne, SVD
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold

# Set the IPython shell interactivity to show all outputs
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
import pandas as pd

# Specify the path to the data JSON file
json_file_path = '/content/drive/MyDrive/CSE272/automotive.json'

# Set the chunk size (adjust as per the memory constraints)
chunk_size = 10000

# Create an empty list to store the chunks of data
data_chunks = []

# Read the JSON file in chunks
for chunk in pd.read_json(json_file_path, lines=True, chunksize=chunk_size):
    # Process each chunk as needed
    data_chunks.append(chunk)

# Concatenate the data chunks into a single DataFrame
data_df = pd.concat(data_chunks, ignore_index=True)

In [ ]:
data_df.head()

,image,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
0,[https://images-na.ssl-images-amazon.com/image...,4,3.0,True,"08 12, 2015",A3ELV1VUCZZGL4,0209688726,{'Color:': ' CA'},AnaG,it's fine. I just would like the stickers to b...,Serve its purpose,1439337600
1,NaN,2,NaN,True,"08 12, 2015",A2FD3FZ2WV6KHQ,0209688726,{'Color:': ' T'},Obi John Kenobi,took me three returns to get one that didn't w...,and it's still not super stable but whatever,1439337600
2,[https://images-na.ssl-images-amazon.com/image...,2,12.0,True,"08 11, 2015",A39NRKABAVSVSE,0209688726,{'Color:': ' AC'},MuddyRiverGirl,While the product is fine the description and ...,Beware! Description is wrong.,1439251200
3,NaN,5,NaN,True,"07 29, 2015",AQSBL5KODHSGW,0209688726,{'Color:': ' T'},Nate S,It's beautiful and blends right in with my woo...,"Awesome desk stand, simple and elegant",1438128000
4,NaN,5,NaN,False,"07 11, 2015",AU45OQPK2J5JF,0209688726,{'Color:': ' T'},Kort,I love this stand! I had been looking around f...,I love this stand,1436572800


In [ ]:
data_df_renamed = data_df.rename(columns={
    "reviewerID": "User ID",
    "asin": "Product ID",
    "overall": "Rating",
    "reviewText": "Review",
    "summary": "Title",
    "unixReviewTime": "Timestamp"
})
data_df_info = data_df_renamed[["User ID", "Product ID", "Rating", "Review", "Title"]]

In [ ]:
data_df_info.to_csv('/content/drive/MyDrive/CSE272/data_parsed.csv',index=None)

In [ ]:
data_df_info

,User ID,Product ID,Rating,Review,Title
0,A3ELV1VUCZZGL4,0209688726,4,it's fine. I just would like the stickers to b...,Serve its purpose
1,A2FD3FZ2WV6KHQ,0209688726,2,took me three returns to get one that didn't w...,and it's still not super stable but whatever
2,A39NRKABAVSVSE,0209688726,2,While the product is fine the description and ...,Beware! Description is wrong.
3,AQSBL5KODHSGW,0209688726,5,It's beautiful and blends right in with my woo...,"Awesome desk stand, simple and elegant"
4,AU45OQPK2J5JF,0209688726,5,I love this stand! I had been looking around f...,I love this stand
...,...,...,...,...,...
7990161,A1M8ITTFK4PRD6,B01HJI17Y8,5,I installed them on my 135i and it was a pain ...,Works great but pain to install.
7990162,ACDJQSK6FZIXX,B01HJI17Y8,1,Both bulbs blew after one month of use. Dont w...,TRASH
7990163,AMEQAMPAF1T59,B01HJI17Y8,3,"I guess they are okay for the price, bur if yo...",They are okay!
7990164,AF67UV2T2TFWB,B01HJI17Y8,5,Great product fast shipping and awesome commun...,Five Stars


In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize VADER
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# Example review
review = "sucks"

# Perform sentiment analysis
sentiment_scores = sia.polarity_scores(review)

# Get the sentiment score
compound_score = sentiment_scores['compound']

# Print the sentiment score
print("Sentiment Score:", compound_score)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

Sentiment Score: 0.0516


In [ ]:
data_df_info_demo = data_df_info.head(10)

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm import tqdm

# Initialize VADER
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# Convert float values to strings
data_df_info['Review'] = data_df_info['Review'].astype(str)
data_df_info['Title'] = data_df_info['Title'].astype(str)

# Define function to calculate sentiment score
def calculate_sentiment_score(row):
    return sia.polarity_scores(row['Title'])['compound']

# Apply sentiment analysis with progress bar
tqdm.pandas(desc="Calculating Sentiment Score")
data_df_info['Sentiment Score'] = data_df_info.progress_apply(calculate_sentiment_score, axis=1)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

<ipython-input-28-b53b783def06>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df_info['Review'] = data_df_info['Review'].astype(str)
<ipython-input-28-b53b783def06>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df_info['Title'] = data_df_info['Title'].astype(str)
Calculating Sentiment Score: 100%|██████████| 7990166/7990166 [13:58<00:00, 9528.57it/s] 
<ipython-input-28-b53b783def06>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [ ]:
data_df_info.to_csv('/content/drive/MyDrive/CSE272/data_parsed_sentiment_with_title.csv',index=None)

In [ ]:
data_df_info = pd.read_csv('/content/drive/MyDrive/CSE272/data_parsed_sentiment_with_title.csv')

In [ ]:
data_df_info = data_df_info[['User ID','Product ID','Rating']]
column_names = ['reviewerID', 'asin', 'overall']
data_df_info.columns = column_names

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(data_df_info[['reviewerID','asin','overall']],reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2,random_state=7)

In [ ]:
algo = SVD()
algo.fit(trainset)

In [ ]:

import pickle

predictions = algo.test(testset)

# Save the predictions to a file using pickle
with open("/content/drive/MyDrive/CSE272/Data/predictions.pkl", "wb") as f:
    pickle.dump(predictions, f)


In [ ]:
predictions

[Prediction(uid='AXFAL035JF7XY', iid='B00LO3RX8E', r_ui=3.0, est=4.059221581549874, details={'was_impossible': False}),
 Prediction(uid='A71DY9BDFX953', iid='B00ZEJGDT4', r_ui=5.0, est=4.665464355302288, details={'was_impossible': False}),
 Prediction(uid='AKHWM1II23PNW', iid='B003SKDGEQ', r_ui=5.0, est=4.822398308903312, details={'was_impossible': False}),
 Prediction(uid='A14HK7SPPXBLEG', iid='B00HJEUUKU', r_ui=2.0, est=3.1200719341525236, details={'was_impossible': False}),
 Prediction(uid='A12EPFM0K6WVDP', iid='B000VKRQ8I', r_ui=5.0, est=4.906684341977959, details={'was_impossible': False}),
 Prediction(uid='AYMT4VKP319GM', iid='B009SSV26Q', r_ui=5.0, est=3.3198201361755997, details={'was_impossible': False}),
 Prediction(uid='A2IGZLYJYZD070', iid='B01DYE5H9C', r_ui=5.0, est=4.043594097122105, details={'was_impossible': False}),
 Prediction(uid='A3SBDHOJ3TFLFV', iid='B00KTHXO2U', r_ui=4.0, est=3.903352558112088, details={'was_impossible': False}),
 Prediction(uid='A2Z7KRMD1WBHPV', 

In [ ]:
accuracy.mae(predictions, verbose=True)

MAE:  0.8825


0.882489867922826

In [ ]:
accuracy.rmse(predictions, verbose=True)

RMSE: 1.1974


1.197449823682455

In [ ]:
import warnings; warnings.simplefilter('ignore')

Save Top-K recommendations

In [ ]:
top_k_recommendations = defaultdict(list)

In [ ]:
import pandas as pd
from tqdm import tqdm

# Create an empty DataFrame to store the recommendations
recommendations_df = pd.DataFrame(columns=['User ID', 'Item ID', 'Predicted Rating'])

# Create a dictionary to store the top k recommendations for each user
top_k_recommendations = defaultdict(list)

# Iterate over the predictions and store the top k recommendations for each user
for prediction in tqdm(predictions, desc='Generating Recommendations'):
    user_id = prediction.uid
    item_id = prediction.iid
    rating = prediction.est
    
    # Append the item and its predicted rating to the user's recommendations list
    top_k_recommendations[user_id].append((item_id, rating))
    
k = 10

# Sort the recommendations for each user by the predicted rating in descending order
for user_id in top_k_recommendations:
    top_k_recommendations[user_id].sort(key=lambda x: x[1], reverse=True)

# Convert the top_k_recommendations dictionary to a DataFrame
top_k_df = pd.DataFrame([(user_id, recommendations[:k]) for user_id, recommendations in top_k_recommendations.items()], columns=['User ID', 'Recommendations'])

# Explode the Recommendations column to create rows for each recommendation
top_k_df = top_k_df.explode('Recommendations')

# Split the Recommendation column into separate columns
top_k_df[['Item ID', 'Predicted Rating']] = pd.DataFrame(top_k_df['Recommendations'].tolist(), index=top_k_df.index)

# Drop the original Recommendations column
top_k_df.drop('Recommendations', axis=1, inplace=True)

# Append the recommendations to the recommendations_df DataFrame
recommendations_df = recommendations_df.append(top_k_df, ignore_index=True)

# Print the recommendations DataFrame
print(recommendations_df)


Generating Recommendations: 100%|██████████| 1598034/1598034 [00:17<00:00, 89336.43it/s] 


                User ID     Item ID  Predicted Rating
0         AXFAL035JF7XY  B00LO3RX8E          4.059222
1         A71DY9BDFX953  B00ZEJGDT4          4.665464
2         AKHWM1II23PNW  B003SKDGEQ          4.822398
3         AKHWM1II23PNW  B002YIAC6I          4.607499
4         AKHWM1II23PNW  B01012DGYI          4.605803
...                 ...         ...               ...
1589677  A1EUWU0U0YPAHT  B00514A2KS          4.361221
1589678  A2FLNG9834PA2A  B0080DLDA0          4.496069
1589679  A3EYHBNQ5PP8T5  B008K7ZIFW          3.760430
1589680  A1MJNDZKNXRU2L  B00GO46L54          4.183253
1589681  A18WB0AH58NKAB  B000CNAOKQ          3.749969

[1589682 rows x 3 columns]


In [ ]:
top_k_recommendations['A10C1QUYVVN7FK']

[('B01BLSL6PM', 4.039474714600151)]

In [ ]:
recommendations_df.to_csv('/content/drive/MyDrive/CSE272/recommendation_results.csv',index=None)

In [ ]:
recommendations_df = pd.read_csv('/content/drive/MyDrive/CSE272/recommendation_results.csv')

In [ ]:
recommendations_df.head(1000).groupby('User ID').apply(lambda x: pd.DataFrame(x.values, columns=recommendations_df.columns)).iloc[:, 1:]

Item ID Predicted Rating
User ID                                            
A0362231G0D49Y5H52ES 0  B000E5U3ZS         4.405707
A0473259F6GQNBD88IYN 0  B00JBS1NQ2              5.0
                     1  B00QIM9HS0              5.0
                     2  B0002ILJYQ              5.0
                     3  B00J8DPFKA         4.941579
...                            ...              ...
AZU394JNIV8FM        1  B00N713YMA         4.927045
                     2  B000C541F8         4.791072
                     3  B000C544X2         4.682261
                     4  B000C30LQ8         4.553162
AZUFUFK5PVCUE        0  B00MG687JW         4.181732

[1000 rows x 2 columns]

In [ ]:
import pickle
with open('/content/drive/MyDrive/CSE272/Data/trainset.pkl', 'wb') as file:
    pickle.dump(trainset, file)

In [ ]:
# Load the pickled object from the file
with open('/content/drive/MyDrive/CSE272/Data/testset.pkl', 'rb') as file:
    testset = pickle.load(file)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
top_k_df = pd.read_csv('/content/drive/MyDrive/CSE272/recommendation_results.csv')

In [ ]:
# reference: https://github.com/NicolasHug/Surprise/blob/master/examples/precision_recall_at_k.py

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

def f_measure(precisions, recalls):
    precision = sum(prec for prec in precisions.values()) / len(precisions)
    recall = sum(rec for rec in recalls.values()) / len(recalls)
    return 2*precision*recall / (precision + recall)

In [ ]:
precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=3.5)

In [ ]:
print("Precision:", sum(prec for prec in precisions.values()) / len(precisions))
print("Recall:", sum(rec for rec in recalls.values()) / len(recalls))
print("F-measure:", f_measure(precisions, recalls))

Precision: 0.7689322649766035
Recall: 0.7833193921937336
F-measure: 0.7760591546574566


In [ ]:
# Define the conversion criteria or threshold
conversion_threshold = 4.0  

# Compute the conversion rate for the top-k predictions
k = 10  # Example: Consider the top 10 predictions
total_users = len(predictions)
converted_users = 0

# Iterate over the predictions and count the number of users with at least one conversion in the top-k predictions
for user_id, user_predictions in top_k_recommendations.items():
    top_k_ratings = [rating for item_id, rating in user_predictions[:k]]
    if any(rating >= conversion_threshold for rating in top_k_ratings):
        converted_users += 1

# Compute the conversion rate
conversion_rate = converted_users / total_users

# Print the conversion rate
print(f"Conversion Rate: {conversion_rate * 100:.2f}%")

Conversion Rate: 59.57%


In [ ]:
from collections import defaultdict
import pandas as pd
from tqdm import tqdm

# Create a dictionary to store the ground truth items for each user
ground_truth = defaultdict(set)

# Iterate over the test set to populate the ground truth dictionary
for user_id, item_id, rating in testset:
    ground_truth[user_id].add(item_id)

# Compute the number of relevant items, recommended items, and correctly recommended items for each user
num_users = len(top_k_df['User ID'].unique())
num_relevant = 0
num_recommended = 0
num_correct = 0

import pandas as pd

# Compute the number of relevant items, recommended items, and correctly recommended items for each user
grouped_df = top_k_df.groupby('User ID')

num_relevant = grouped_df['Item ID'].apply(set).apply(len).sum()
num_recommended = grouped_df['Item ID'].apply(len).sum()
num_correct = grouped_df.apply(lambda x: set(x['Item ID']).intersection(ground_truth[x['User ID'].iloc[0]])).apply(len).sum()

# Compute precision, recall, and F-measure
precision = num_correct / num_recommended if num_recommended > 0 else 0
recall = num_correct / num_relevant if num_relevant > 0 else 0
f_measure = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

print("Precision:", precision)
print("Recall:", recall)
print("F-measure:", f_measure)


Precision: 0.9955399885008448
Recall: 1.0
F-measure: 0.9977650102103413


In [ ]:
import pandas as pd

def precision_recall_at_k(top_k_df, testset, k=10, threshold=3.5):
    ground_truth = defaultdict(set)

    for user_id, item_id, rating in testset:
        ground_truth[user_id].add(item_id)

    top_k_df['True Rating'] = top_k_df.apply(lambda x: item_in_ground_truth(x['User ID'], x['Item ID'], ground_truth), axis=1)

    df_grouped = top_k_df.groupby('User ID')

    n_rel = df_grouped['True Rating'].apply(lambda x: sum(x >= threshold))
    n_rec_k = df_grouped['Item ID'].apply(lambda x: min(k, len(x)))

    n_rel_and_rec_k = df_grouped.apply(lambda x: sum((x['True Rating'] >= threshold) & (x['Item ID'].rank(method='first') <= k)))

    precisions = n_rel_and_rec_k / n_rec_k
    recalls = n_rel_and_rec_k / n_rel

    precisions.fillna(0, inplace=True)
    recalls.fillna(0, inplace=True)

    return precisions, recalls

def item_in_ground_truth(user_id, item_id, ground_truth):
    return item_id in ground_truth[user_id]

precisions, recalls = precision_recall_at_k(top_k_df, testset)

final_precision = precisions.mean()
final_recall = recalls.mean()
final_f_measure = 2 * (final_precision * final_recall) / (final_precision + final_recall)


print("Precision:", final_precision)
print("Recall:", final_recall)
print("F-measure:", final_f_measure)


Precision: 0.0
Recall: 0.0
F-measure: nan


In [ ]:
from surprise import accuracy

# Compute the recall for the top-10 recommendations
recall = accuracy.recall_at_k(predictions, k=10, threshold=4.0)

print("Recall at K:", recall)
